In [14]:
import numpy as np

# 11.1

In [7]:
# vertices
V = set(range(10))
# N[v]: neighbors of v
N = [
    {2,4,7}, 
    {2,3,4,6},
    {0,1,3,6,7,8},
    {1,2},
    {0,1,5,6},
    {4,9},
    {1,2,4,7,8},
    {0,2,6,8},
    {2,6,7},
    {5}
]

letter = list("abcdefghij")

def string(S):
    if S:
        return "".join(([letter[v] for v in sorted(S)]))
    else:
        return "-"

# Bron Kerbosch Alg.
def find_all_maximal_cliques(R, P, X, depth=""):
    if not P and not X:
        clique = [letter[v] for v in sorted(R)]
#         print depth, string(R) or "-", string(P) or "-", string(X) or "-", "    =>", "".join(clique)
        print("".join(clique))
#     else:
#         print depth, string(R) or "-", string(P) or "-", string(X) or "-"

    for v in P:
        find_all_maximal_cliques(R | {v}, P & N[v], X & N[v], depth+" |  ")
        P = P - {v}
        X = X | {v}

    return

print "exercise 11.1"
find_all_maximal_cliques(set(), V, set())

print "\nmoral graph"
V = set(range(5))
N = [{1,2}, {0,2}, {0,1,3,4}, {2}, {2}]
find_all_maximal_cliques(set(), V, set())

exercise 11.1
ach
ae
bcd
bcg
beg
cghi
ef
fj

moral graph
abc
cd
ce


$C_1: A,C,H $  
$C_2: A,E $  
$C_3: B,C,D $  
$C_4: B,C,G $  
$C_5: B,G,E $  
$C_6: C,G,H,I$  
$C_7: E,F $    
$C_8: F,J $    

# 11.2

## a) moral graph of the given DAG
<img src="files/moral_graph.png">

## b)
<img src="files/decomposition.jpg">

# 11.3

<img src='103a.png' />

The implementation captures the graph structure in the joint probability formula. Corresponding probabilities are saved in lists/matrices.

In [166]:
# the graph in (a)
def p(b=[0,1],e=[0,1],a=[0,1],r=[0,1]):
    if isinstance(b, int): b = [b]
    if isinstance(e, int): e = [e]
    if isinstance(a, int): a = [a]
    if isinstance(r, int): r = [r]
   
    # variable syntax: p_...x := p(x|...)
    p_b = [0.99, 0.01]
    p_e = [1.-10e-6, 10e-6]
    p_er = [[1.0, 0.0], [0.0, 1.0]]
    p_bea = [[[0.999, 0.001], [0.59, 0.41]], [[0.05, 0.95], [0.02, 0.98]]]
 
    p_sum = 0.0
    for bi in b:
        for ei in e:
            for ai in a:
                for ri in r:
                    p_sum += p_b[bi] * p_e[ei] * p_er[ei][ri] * p_bea[bi][ei][ai]

    return p_sum

def example():
    print "p(b=1) =", p(b=1)
    print "p(b=1|e=1) =", p(b=1,e=1) / p(e=1)
    print "p(b=1|e=1,a=1) =", p(b=1,e=1,a=1) / p(e=1,a=1)
    print
    
    for a in [0,1]: print "p(a=%i) == %f" % (a, p(a=a))
    for b in [0,1]: print "p(b=%i) == %f" % (b, p(b=b))
    for e in [0,1]: print "p(e=%i) == %f" % (e, p(e=e))
    for r in [0,1]: print "p(r=%i) == %f" % (r, p(r=r))
    return
example()

p(b=1) = 0.01
p(b=1|e=1) = 0.01
p(b=1|e=1,a=1) = 0.0235746932884

p(a=0) == 0.989506
p(a=1) == 0.010494
p(b=0) == 0.990000
p(b=1) == 0.010000
p(e=0) == 0.999990
p(e=1) == 0.000010
p(r=0) == 0.999990
p(r=1) == 0.000010


This does the same using the bayesian-belief-networks library. The graph structure is captured by probability functions with globally consistent variable names. The random variables are discrete, same as above.

In [2]:
from bayesian.bbn import build_bbn
from bayesian.utils import make_key

def f_burglary(burglary):
    if burglary:
        return 0.01
    else:
        return 0.99

def f_earthquake(earthquake):
    if earthquake:
        return 10e-6
    else:
        return 1.-10e-6

def f_alarm(burglary, earthquake, alarm):
    table = dict()
    table['fff'] = 0.999
    table['fft'] = 0.001
    table['ftf'] = 0.59
    table['ftt'] = 0.41
    table['tff'] = 0.05
    table['tft'] = 0.95
    table['ttf'] = 0.02
    table['ttt'] = 0.98
    return table[make_key(burglary, earthquake, alarm)]

def f_radio(earthquake, radio):
    table = dict()
    table['ff'] = 1.0
    table['ft'] = 0.0
    table['tf'] = 0.0
    table['tt'] = 1.0
    return table[make_key(earthquake, radio)]


g = build_bbn(
    f_burglary,
    f_earthquake,
    f_alarm,
    f_radio)
g.q()
g.q(alarm=1,radio=1)
print p(b=1,a=1,r=1) / p(a=1,r=1)

+------------+-------+----------+
| Node       | Value | Marginal |
+------------+-------+----------+
| alarm      | False | 0.989506 |
| alarm      | True  | 0.010494 |
| burglary   | False | 0.990000 |
| burglary   | True  | 0.010000 |
| earthquake | False | 0.999990 |
| earthquake | True  | 0.000010 |
| radio      | False | 0.999990 |
| radio      | True  | 0.000010 |
+------------+-------+----------+
+------------+-------+----------+
| Node       | Value | Marginal |
+------------+-------+----------+
| alarm      | False | 0.000000 |
| alarm*     | True* | 1.000000 |
| burglary   | False | 0.976425 |
| burglary   | True  | 0.023575 |
| earthquake | False | 0.000000 |
| earthquake | True  | 1.000000 |
| radio      | False | 0.000000 |
| radio*     | True* | 1.000000 |
+------------+-------+----------+


NameError: name 'p' is not defined

# 11.4

## a) potential functions and the clique-marginal representation

- potential functions are used to represent joint probabilities of cliques, can be easily updated by new data
- they are strictly positive, real-valued and only depend on random variables in their assigned clique
- don't need to be normalised so that $\;\psi_n(C_n) \propto P(C_n)$
- link between joint probability distribution and clique-marginal representation:  

$$P(V \in \mathcal{V})\;=\;\prod_{V \in \mathcal{V}} P(V|V_{pa(V)})\;=\;\frac{\prod_i P(C_i)}{\prod_j P(S_j)}\;=\; \alpha \frac{\prod_i \psi_i(C_i)}{\prod_j \psi_j(S_j)}
$$

## b) exploitation of potential functions to make inferences

1) update clique potential

- initialisation: set all potentials to $1$, multiply nodes $x_k$ with $P(x_k\; |\; parents(x_k))$
- inference by observed evidence at one node: $V_l=v_l$
- introduce indicator function $E(V_l) = \begin{cases}
                                                  1 &\text{ $V_l = v_l$}\\
                                                  0 &\text{ else}
                                                  \end{cases}$
- update clique potential of observed node: $\psi^*_{i}(C_i)=\psi_{i}(C_i)E(V_l)$

2) message passing

- marginal seperator potential (eleminate nodes not in $S_s$): $\; \psi_s^*(S_s)\;=\; \sum_{V \in C_i, V \notin \psi_s} \psi_i(V \in C_i) $
- the update of one clique demands updates of the neighbor cliques which happens by passing the update ratio $\frac{\psi_s^*(S_c)}{\psi_s(S_c)}$ ($S_c$: common seperator)
- updated neighbor potential obtained by multiplication of current potential with the update rate

## c) benefits in comparison to DAG factorisation

- simpler and clearer models due to substitution of subgraphs by cliques
- highly improved interference